<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

This project is for the Applied Data Science Capstone that explores and clusters the neighborhoods in Toronto, Canada.

In [14]:
import numpy as np
import pandas as pd

import urllib.request
#import dryscrape

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c anaconda beautifulsoup4   #uncomment if installing for the first time

from bs4 import BeautifulSoup

Download list of postal codes of Canada using the URL

In [15]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table = pd.read_html(url)

Then, convert the downloaded list to dataframe

In [16]:
neighborhoods = table[0]
neighborhoods

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


We only need to process the cells that have an assigned borough. Hence, remove data with Borough column equals to "Not assigned"

In [17]:
neighborhoods2 = neighborhoods[neighborhoods.Borough != 'Not assigned']
neighborhoods2

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


More than one neighborhood can exist in one postal code area. If exist, they needed to be combined into one row with the neighborhoods separated with a comma.
So, determine if there are duplicates in postal code.

In [18]:
duplicates = neighborhoods2[neighborhoods2.duplicated(['Postal Code'])]
print(duplicates)

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


As showed in the result above, there seem to be no duplicates in postal code.

Now, if a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough. The code below will do this.

In [19]:
neighborhoods2.loc[(neighborhoods2.Borough!="Not assigned") & (neighborhoods2.Neighbourhood=="Not assigned"), 'Neighbourhood'] = neighborhoods2.Borough
neighborhoods2

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Lastly, print the number of rows and columns of the final dataframe.

In [20]:
neighborhoods2.shape

(103, 3)